In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!kill -9 -1

In [2]:
cd /content/drive/My\ Drive

/content/drive/My Drive


In [0]:
from models import Multimobilenet,Multialexnet

In [0]:
from Dataset import train_dataset

In [0]:
from torch.utils.data import DataLoader

In [0]:
from Utils import *

In [0]:
import pandas as pd
df=pd.read_csv('Data_Entry_2017.csv')


In [0]:
images=df['Image Index']
labels=df['Finding Labels']

In [0]:
test_images=[]
test_labels=[]

In [0]:
train_images=images[:4000]
train_labels=labels[:4000]
test_images[0:]=images[4000:]
test_labels[0:]=labels[4000:]

In [0]:
train_images,train_labels=get_existing_images('/content/drive/My Drive/images_001/images',train_images,train_labels)
test_images,test_labels=get_existing_images('/content/drive/My Drive/images_001/images',test_images,test_labels)

In [12]:
encoder=encode_labels(labels)

15


In [13]:
decoder={}
for keys in encoder.keys():
  decoder[encoder[keys]]=keys
decoder[3]

'No Finding'

In [0]:
transform=image_transform(250,225)

In [0]:
import torch
import torchvision
import numpy as np
from torchvision import transforms,datasets
from torch import nn,optim
import torch.nn.functional as F
from torch.utils.data import sampler
from torch.utils.data.sampler import SubsetRandomSampler

In [0]:
train_batch_size=100
test_batch_size=2

In [0]:
trainingset = train_dataset(Labels=train_labels,images=train_images ,encoder=encoder,dataset='/content/drive/My Drive/images_001/images',transform=transform)
trainloader = DataLoader(trainingset,batch_size = train_batch_size,shuffle = True)
testset = train_dataset(Labels=test_labels,images=test_images ,encoder=encoder,dataset='/content/drive/My Drive/images_001/images',transform=transform)
testloader = DataLoader(testset,batch_size = test_batch_size,shuffle = True)


In [18]:
!pip install tensorboardX --no-cache-dir

     |████████████████████████████████| 204kB 28.6MB/s 


In [0]:
from tensorboardX import SummaryWriter
writer = SummaryWriter()

In [0]:
N_epochs = 10
model = Multimobilenet()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

In [0]:
loss=nn.BCEWithLogitsLoss(reduction='mean') # loss function

In [0]:
from glob import glob
def get_most_recent_checkpoint(checkpoint_dir):
    checkpoint_paths = [path for path in glob("{}/model_epoch_*.pth".format(checkpoint_dir))]
    idxes = [int(os.path.basename(path).split('_')[2].split('.')[0]) for path in checkpoint_paths]

    max_idx = max(idxes)
    latest_checkpoint = os.path.join(checkpoint_dir, "model_epoch_{}.pth".format(max_idx))
    print(" [*] Found latest checkpoint: {}".format(latest_checkpoint))
    return latest_checkpoint, max_idx

In [0]:

def checkpoint(epoch):
  model_out_path = "/content/drive/My Drive/ckpt/" + "mobilenet" + "/model_epoch_{}.pth".format(epoch)
  '''
    Model Implementation
    elif opt.model == "Model_Name":
        model_out_path = "ckpt/" + "Model_Name" + "/model_epoch_{}.pth".format(epoch)
  '''
  print(model_out_path)
  torch.save({
            'model': model.state_dict(),
            }, model_out_path)
  print("Checkpoint saved to {}".format(model_out_path))

In [0]:
if len(next(os.walk('/content/drive/My Drive/ckpt/mobilenet'))[2]) != 0:
  
  min_iter = 0
  last_ckpt, min_iter = get_most_recent_checkpoint('/content/drive/My Drive/ckpt/mobilenet')
  min_iter=min_iter+1
  print('Getting most recent checkpoint -------->')
  checkpoint=torch.load(last_ckpt)
  model.load_state_dict(checkpoint['model'])
else:
  min_iter=0

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
model=model.to(device)

In [0]:
%pycat models.py

In [38]:
%%writefile models.py
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import torch.nn.functional as F
from torch import nn

class Multimobilenet(nn.Module):
    def __init__(self):
        super().__init__()
        self.base_model = models.mobilenet_v2().features  # take the model without classifier
        last_channel = models.mobilenet_v2().last_channel # size of the layer before the classifier

        # the input for the classifier should be two-dimensional, but we will have
        # [<batch_size&gt;, <channels&gt;, <width&gt;, <height&gt;]
        # so, let's do the spatial averaging: reduce <width&gt; and <height&gt; to 1
        self.pool = nn.AdaptiveAvgPool2d((1, 1))

        # create separate classifiers for our outputs
        self.Type = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=last_channel, out_features=2)
        )
        self.target = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=last_channel, out_features=15)
        )
    def forward(self, x):
        x = self.base_model(x)
        x = self.pool(x)
        
      # reshape from [batch, channels, 1, 1] to [batch, channels] to put it into classifier
        x = torch.flatten(x, start_dim=1)
        
        return {
          'Type':F.softmax(self.Type(x)),
          'targets':F.softmax(self.target(x))
         }



class Multialexnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.base_model = models.alexnet()  # take the model without classifier
                # the input for the classifier should be two-dimensional, but we will have
        # [<batch_size&gt;, <channels&gt;, <width&gt;, <height&gt;]
        # so, let's do the spatial averaging: reduce <width&gt; and <height&gt; to 1
        self.pool = nn.AdaptiveAvgPool2d((1, 1))

        # create separate classifiers for our outputs
        self.Type = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=1000, out_features=2)
        )
        self.target = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=1000, out_features=15)
        )
    def forward(self, x):
        #print('Passing 1st phase')
        x = self.base_model(x)
        #print('Passing 2nd phase')
        #x = self.pool(x)

        # reshape from [batch, channels, 1, 1] to [batch, channels] to put it into classifier
        #print('Flattening')
        #x = torch.flatten(x, start_dim=1)

        return {
          'Type': F.softmax(self.Type(x)),
          'targets': F.softmax(self.target(x))
        }

Overwriting models.py


In [0]:
def accuracy_evaluator(pred_labels, true_labels):
    pred_labels = torch.ByteTensor(pred_labels)
    true_labels = torch.ByteTensor(true_labels)

    assert 1 >= pred_labels.all() >= 0
    assert 1 >= true_labels.all() >= 0

    # True Positive (TP): we predict a label of 1 (positive), and the true label is 1.
    TP = torch.sum((pred_labels >= 0.7) & ((true_labels == 1)))

    # True Negative (TN): we predict a label of 0 (negative), and the true label is 0.
    TN = torch.sum((pred_labels < 0.7) & (true_labels == 0))

    # False Positive (FP): we predict a label of 1 (positive), but the true label is 0.
    FP = torch.sum((pred_labels >= 0.7) & (true_labels == 0))

    # False Negative (FN): we predict a label of 0 (negative), but the true label is 1.
    FN = torch.sum((pred_labels < 0.7) & (true_labels == 1))
    return TP+TN-FP-FN

In [0]:
batch=next(iter(testloader))
img=batch['img']
img=img.to(device)
target_labels=batch['labels']
output=model(img)
loss_type=loss(output['Type'].to(device),target_labels['Type'].to(device))
loss_target=loss(output['targets'].to(device),target_labels['targets'].to(device))
testloss_type= loss_type.item()
testloss_target=loss_target.item()
accuracy_for_type=accuracy_evaluator(output['Type'],target_labels['Type'])/test_batch_size
accuracy_for_target=accuracy_evaluator(output['targets'],target_labels['targets'])/test_batch_size

In [36]:
for epoch in range(min_iter, N_epochs + 1):
  total_loss=0
  for k,batch in enumerate(trainloader):
    optimizer.zero_grad()
    img = batch['img']
    img=img.to(device)
    target_labels = batch['labels']
    output = model(img)
    loss_type=loss(output['Type'].to(device),target_labels['Type'].to(device))
    loss_target=loss(output['targets'].to(device),target_labels['targets'].to(device))
    curr_loss= loss_type.item()+loss_target.item()
    total_loss+=curr_loss
    loss_type.backward(retain_graph=True)
    loss_target.backward()
    optimizer.step()
    writer.add_scalar('Multi Class Loss',loss_type.item())
    writer.add_scalar('Multi Label Loss',loss_target.item())
    print('completed batch',k+1,' Total number of batches : ',len(trainloader),' Class Loss = ',loss_type.item(),'Label Loss = ',loss_target.item())
  print('Done for epoch ',epoch+1)
  print('loss for epoch = ',total_loss/len(trainloader))
  checkpoint(epoch)
  testloss_type=0
  testloss_target=0
  accuracy_for_type=0
  accuracy_for_target=0
  for k,batch in enumerate(testloader):
    img=batch['img']
    img=img.to(device)
    target_labels=batch['labels']
    output=model(img)
    loss_type=loss(output['Type'].to(device),target_labels['Type'].to(device))
    loss_target=loss(output['targets'].to(device),target_labels['targets'].to(device))
    testloss_type+= loss_type.item()
    testloss_target+=loss_target.item()
    accuracy_for_type+=accuracy_evaluator(output['Type'],target_labels['Type'])/test_batch_size
    accuracy_for_target+=accuracy_evaluator(output['targets'],target_labels['targets'])/test_batch_size
  print('Accuracy for Normal/Abnormal Class : ',accuracy_for_type/(len(testloader)))
  print('Accuracy for Labels : ',accuracy_for_target/len(testloader))


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


completed batch 1  Total number of batches :  40  Class Loss =  0.6934219002723694 Label Loss =  0.6933416128158569
completed batch 2  Total number of batches :  40  Class Loss =  0.6925032138824463 Label Loss =  0.6929797530174255
completed batch 3  Total number of batches :  40  Class Loss =  0.6869975328445435 Label Loss =  0.6932076811790466
completed batch 4  Total number of batches :  40  Class Loss =  0.6974266171455383 Label Loss =  0.693162739276886
completed batch 5  Total number of batches :  40  Class Loss =  0.6931918263435364 Label Loss =  0.6931448578834534
completed batch 6  Total number of batches :  40  Class Loss =  0.6931365728378296 Label Loss =  0.6931506991386414
completed batch 7  Total number of batches :  40  Class Loss =  0.6931406855583191 Label Loss =  0.6931411027908325
completed batch 8  Total number of batches :  40  Class Loss =  0.6931484341621399 Label Loss =  0.693150520324707
completed batch 9  Total number of batches :  40  Class Loss =  0.69314312

KeyboardInterrupt: ignored